In [1]:
import logging
from sys import stdout
from datetime import datetime
import numpy as np

logging.basicConfig(
    level=logging.INFO,
    format="[%(asctime)s][%(levelname)s] %(message)s",
    stream=stdout,
    datefmt="%m-%d %H:%M:%S",
)
logging.info(datetime.now())

# ####################################################################
def eliminacion_gaussiana(A: np.ndarray) -> np.ndarray:
    """Resuelve un sistema de ecuaciones lineales mediante el método de eliminación gaussiana.

    ## Parameters

    ``A``: matriz aumentada del sistema de ecuaciones lineales. Debe ser de tamaño n-by-(n+1), donde n es el número de incógnitas.

    ## Return

    ``solucion``: vector con la solución del sistema de ecuaciones lineales.

    """
    if not isinstance(A, np.ndarray):
        logging.debug("Convirtiendo A a numpy array.")
        A = np.array(A)
    assert A.shape[0] == A.shape[1] - 1, "La matriz A debe ser de tamaño n-by-(n+1)."
    n = A.shape[0]

    for i in range(0, n - 1):  # loop por columna

        # --- encontrar pivote
        p = None  # default, first element
        for pi in range(i, n):
            if A[pi, i] == 0:
                # must be nonzero
                continue

            if p is None:
                # first nonzero element
                p = pi
                continue

            if abs(A[pi, i]) < abs(A[p, i]):
                p = pi

        if p is None:
            # no pivot found.
            raise ValueError("No existe solución única.")

        if p != i:
            # swap rows
            logging.debug(f"Intercambiando filas {i} y {p}")
            _aux = A[i, :].copy()
            A[i, :] = A[p, :].copy()
            A[p, :] = _aux

        # --- Eliminación: loop por fila
        for j in range(i + 1, n):
            m = A[j, i] / A[i, i]
            A[j, i:] = A[j, i:] - m * A[i, i:]

        logging.info(f"\n{A}")

    if A[n - 1, n - 1] == 0:
        raise ValueError("No existe solución única.")

        print(f"\n{A}")
    # --- Sustitución hacia atrás
    solucion = np.zeros(n)
    solucion[n - 1] = A[n - 1, n] / A[n - 1, n - 1]

    for i in range(n - 2, -1, -1):
        suma = 0
        for j in range(i + 1, n):
            suma += A[i, j] * solucion[j]
        solucion[i] = (A[i, n] - suma) / A[i, i]

    return solucion

# ####################################################################
def descomposicion_LU(A: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    """Realiza la descomposición LU de una matriz cuadrada A.
    [IMPORTANTE] No se realiza pivoteo.

    ## Parameters

    ``A``: matriz cuadrada de tamaño n-by-n.

    ## Return

    ``L``: matriz triangular inferior.

    ``U``: matriz triangular superior. Se obtiene de la matriz ``A`` después de aplicar la eliminación gaussiana.
    """

    A = np.array(
        A, dtype=float
    )  # convertir en float, porque si no, puede convertir como entero

    assert A.shape[0] == A.shape[1], "La matriz A debe ser cuadrada."
    n = A.shape[0]

    L = np.zeros((n, n), dtype=float)

    for i in range(0, n):  # loop por columna

        # --- deterimnar pivote
        if A[i, i] == 0:
            raise ValueError("No existe solución única.")

        # --- Eliminación: loop por fila
        L[i, i] = 1
        for j in range(i + 1, n):
            m = A[j, i] / A[i, i]
            A[j, i:] = A[j, i:] - m * A[i, i:]

            L[j, i] = m

        logging.info(f"\n{A}")

    if A[n - 1, n - 1] == 0:
        raise ValueError("No existe solución única.")

    return L, A

# ####################################################################
def resolver_LU(L: np.ndarray, U: np.ndarray, b: np.ndarray) -> np.ndarray:
    """Resuelve un sistema de ecuaciones lineales mediante la descomposición LU.

    ## Parameters

    ``L``: matriz triangular inferior.

    ``U``: matriz triangular superior.

    ``b``: vector de términos independientes.

    ## Return

    ``solucion``: vector con la solución del sistema de ecuaciones lineales.

    """

    n = L.shape[0]

    # --- Sustitución hacia adelante
    logging.info("Sustitución hacia adelante")

    y = np.zeros((n, 1), dtype=float)

    y[0] = b[0] / L[0, 0]

    for i in range(1, n):
        suma = 0
        for j in range(i):
            suma += L[i, j] * y[j]
        y[i] = (b[i] - suma) / L[i, i]

    logging.info(f"y = \n{y}")

    # --- Sustitución hacia atrás
    logging.info("Sustitución hacia atrás")
    sol = np.zeros((n, 1), dtype=float)

    sol[-1] = y[-1] / U[-1, -1]

    for i in range(n - 2, -1, -1):
        logging.info(f"i = {i}")
        suma = 0
        for j in range(i + 1, n):
            suma += U[i, j] * sol[j]
        logging.info(f"suma = {suma}")
        logging.info(f"U[i, i] = {U[i, i]}")
        logging.info(f"y[i] = {y[i]}")
        sol[i] = (y[i] - suma) / U[i, i]

    logging.debug(f"x = \n{sol}")
    return sol

# ####################################################################
def matriz_aumentada(A: np.ndarray, b: np.ndarray) -> np.ndarray:
    """Construye la matriz aumentada de un sistema de ecuaciones lineales.

    ## Parameters

    ``A``: matriz de coeficientes.

    ``b``: vector de términos independientes.

    ## Return

    ``a``:

    """
    if not isinstance(A, np.ndarray):
        logging.debug("Convirtiendo A a numpy array.")
        A = np.array(A, dtype=float)
    if not isinstance(b, np.ndarray):
        b = np.array(b, dtype=float)
    assert A.shape[0] == b.shape[0], "Las dimensiones de A y b no coinciden."
    return np.hstack((A, b.reshape(-1, 1)))

# ####################################################################
def separar_m_aumentada(Ab: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    """Separa la matriz aumentada en la matriz de coeficientes y el vector de términos independientes.

    ## Parameters
    ``Ab``: matriz aumentada.

    ## Return
    ``A``: matriz de coeficientes.
    ``b``: vector de términos independientes.
    """
    logging.debug(f"Ab = \n{Ab}")
    if not isinstance(Ab, np.ndarray):
        logging.debug("Convirtiendo Ab a numpy array")
        Ab = np.array(Ab, dtype=float)
    return Ab[:, :-1], Ab[:, -1].reshape(-1, 1)

# ####################################################################
def calc_determinante(A: list[list[float]]) -> float:
    """Función que calcula el determinante usando la descomposición LU.

    ## Parameters
    ``A``: Matriz cuadrada de tamaño n x n

    ## Return
    ``detA``: Determinante de la matriz A

    """
    A = np.array(A)
    L, U = descomposicion_LU(A)
    
    # El determinante es el producto de los elementos diagonales de U
    detA = np.prod(np.diag(U))
    
    return detA

# Ejemplo de uso
A = [
    [-4,  2, -4, -4,  1,  2,  5,  3,  5,  1],
    [ 1,  0,  4,  3,  0, -2,  3,  0,  1,  5],
    [ 5,  5, -4,  5, -4,  2,  2,  2,  4,  4],
    [-1,  3,  4, -1, -4,  0,  5,  0,  0,  5],
    [ 4,  1,  4,  2,  0,  0,  3, -1,  0,  2],
    [ 2, -2,  1, -1, -2, -3,  2, -2,  4, -1],
    [ 3, -2, -3, -2, -1, -3,  5, -1,  5,  0],
    [ 3,  4, -3,  3, -2,  2, -4, -4,  1,  5],
    [-4,  0,  3,  3, -3, -2, -2,  0,  5, -4],
    [-2,  4,  4, -2, -1,  1,  5, -1,  3, -3]
]

detA = calc_determinante(A)
print(f"El determinante de A es: {detA}")


[07-19 21:22:53][INFO] 2024-07-19 21:22:53.225702
[07-19 21:22:53][INFO] 
[[-4.    2.   -4.   -4.    1.    2.    5.    3.    5.    1.  ]
 [ 0.    0.5   3.    2.    0.25 -1.5   4.25  0.75  2.25  5.25]
 [ 0.    7.5  -9.    0.   -2.75  4.5   8.25  5.75 10.25  5.25]
 [ 0.    2.5   5.    0.   -4.25 -0.5   3.75 -0.75 -1.25  4.75]
 [ 0.    3.    0.   -2.    1.    2.    8.    2.    5.    3.  ]
 [ 0.   -1.   -1.   -3.   -1.5  -2.    4.5  -0.5   6.5  -0.5 ]
 [ 0.   -0.5  -6.   -5.   -0.25 -1.5   8.75  1.25  8.75  0.75]
 [ 0.    5.5  -6.    0.   -1.25  3.5  -0.25 -1.75  4.75  5.75]
 [ 0.   -2.    7.    7.   -4.   -4.   -7.   -3.    0.   -5.  ]
 [ 0.    3.    6.    0.   -1.5   0.    2.5  -2.5   0.5  -3.5 ]]
[07-19 21:22:53][INFO] 
[[ -4.     2.    -4.    -4.     1.     2.     5.     3.     5.     1.  ]
 [  0.     0.5    3.     2.     0.25  -1.5    4.25   0.75   2.25   5.25]
 [  0.     0.   -54.   -30.    -6.5   27.   -55.5   -5.5  -23.5  -73.5 ]
 [  0.     0.   -10.   -10.    -5.5    7.   -17.5   


Modificación de la función eliminacion_gaussiana:
Se añadió una verificación para lanzar una excepción si no existe una solución única durante el proceso de eliminación.

Modificación de la función descomposicion_LU:

Se añadió una verificación para lanzar una excepción si no existe una solución única durante el proceso de descomposición LU.
Creación de la función calc_determinante:

Se añadió esta nueva función para calcular el determinante de una matriz cuadrada utilizando la descomposición LU.
En caso de que se lance una excepción (indicando que no existe una solución única), el determinante se establece como 0.0.
Prueba de ejemplo:

Se añadió un ejemplo de uso para calcular el determinante de la matriz 
𝐴
2
A2 proporcionada.
Manejo de excepción:

En la función calc_determinante, se añadió un bloque try-except para capturar la excepción lanzada cuando no existe una solución única y establecer el determinante como 0.0.
